# Experiments on embeddings quantization and SVD

## Configuration & helpers

In [ ]:
import boto3
import pandas as pd
import numpy as np

boto_session = boto3.Session(region_name="us-east-1", profile_name="<INSERT YOUR AWS CONFIG PROFILE>"))
bruntime = boto_session.client('bedrock-runtime')


In [ ]:
def form_analysis_df(df_list, measurement):

    final_df = pd.concat(df_list)
    maxperf = {model_id:final_df.query(f"model_id=='{model_id}' and type=='classic'")[measurement].values[0] for model_id in final_df.model_id.unique()}
    classicsize = {model_id:final_df.query(f"model_id=='{model_id}' and type=='classic'")['size_in_bytes'].values[0] for model_id in final_df.model_id.unique()}
    final_df['log_size']=np.log(final_df.size_in_bytes)
    final_df['size_multiplier']=final_df.apply(lambda x: classicsize[x['model_id']] / x['size_in_bytes'], axis = 1)
    final_df['performance_retention']=final_df.apply(lambda x: round(100*x[measurement] / maxperf[x['model_id']], 2), axis = 1)
    return final_df

## STS-B

In [ ]:
from experiments import STSLabAssistant
sts_titan_operator = STSLabAssistant.instantiate_titan_experiments(bruntime)
titan_df=sts_titan_operator.run()

sts_cohere_operator = STSLabAssistant.instantiate_cohere_experiments(bruntime)
cohere_df=sts_cohere_operator.run()

In [ ]:

final_sts_df=form_analysis_df([titan_df, cohere_df], measurement="spearman")

In [ ]:
import plotly.express as px

fig = px.bar(final_sts_df.query("type!='classic'"), x = "type", y = "performance_retention", color = "model_id", barmode="group", 
       range_y=[final_sts_df['performance_retention'].min()*0.99, final_sts_df['performance_retention'].max()*1.01],
       category_orders={"type":["binary" "rotated-binary", "ternary", "rotated-ternary", "quaternary", "rotated-quaternary", "scalar", "rotated-scalar"]}, 
       labels = {"performance_retention":"Performance retention", "type":"Method", "model_id":"Model"},
       title = "Performance retention - STSB")

fig.add_hline(y=100)

fig.show()

## Banking77

In [ ]:
from experiments import Banking77LabAssistant
banking_operator = Banking77LabAssistant.instantiate_titan_experiments(bruntime)
titan_banking_df=banking_operator.run()

cohere_banking_operator = Banking77LabAssistant.instantiate_cohere_experiments(bruntime)
cohere_banking_df=cohere_banking_operator.run()

In [ ]:
final_banking_df = form_analysis_df([titan_banking_df, cohere_banking_df], measurement="accuracy")

In [ ]:
import plotly.express as px

fig = px.bar(final_banking_df.query("type!='classic'"), x = "type", y = "performance_retention", color = "model_id", barmode="group", 
       range_y=[final_banking_df['performance_retention'].min()*0.99,final_banking_df['performance_retention'].max()*1.01],
       category_orders={"type":["binary", "rotated-binary", "ternary","rotated-ternary",  "scalar","rotated-scalar" ]}, 
       labels = {"performance_retention":"Performance retention", "type":"Method", "model_id":"Model"})

fig.add_hline(y=100)

fig.show()

## Combining both tasks

In [ ]:
df = pd.read_pickle("final_analysis.pickle")

In [ ]:
tmp = final_sts_df.rename({"spearman":"performance"}, axis = 1)
tmp['task']="STS-B"

tmp2 = final_banking_df.rename({"accuracy":"performance"}, axis = 1)
tmp2['task']="Banking77"

df = pd.concat([tmp, tmp2])

In [ ]:
df['quantization']=df.type.apply(lambda x:True if x!="classic" else False)
df["rotation"]=df.type.apply(lambda x:True if "rotated" in x else False)

In [ ]:
import plotly.express as px

px.scatter(df, x = "model_id", y = "performance_retention", size = "size_multiplier", color = "type", symbol = "task")

In [ ]:
import plotly.express as px

px.scatter(df, x = "model_id", y = "performance_retention", size = "size_multiplier", color = "rotation", 
           title = "Performance retention induced by rotation")

In [ ]:
px.scatter(df, x = "size_multiplier", y = "performance_retention", color = "task")

In [ ]:
import plotly.express as px

fig = px.bar(df.query("type!='classic'"), x = "type", y = "performance_retention", color = "model_id", barmode="group", 
       range_y=[df['performance_retention'].min()*0.99, df['performance_retention'].max()*1.01],
       category_orders={"type":["binary" "rotated-binary", "ternary", "rotated-ternary", "scalar", "rotated-scalar"]}, 
       labels = {"performance_retention":"Performance retention", "type":"Method", "model_id":"Model"},
       title = "Performance retention", facet_col= "task")

fig.add_hline(y=100)

fig.show()

In [ ]:
df

In [ ]:
grouped = df.groupby(['type']).agg({"size_multiplier":np.mean,"performance_retention":np.mean}).reset_index()
print(grouped.sort_values(by="performance_retention", ascending=False).to_markdown(index = False))

In [ ]:
display_df = df.drop(["time", "log_size"], axis = 1)[[ "task", "model_id", "type", "dimension", "size_multiplier", "performance_retention"]]

print(display_df.sort_values(by="performance_retention", ascending = False).head(10).to_markdown(index = False))
bb

In [ ]:
print(display_df.sort_values(by="performance_retention", ascending = True).head(10).to_markdown(index = False))


In [ ]:
df.to_pickle("final_analysis.pickle")

In [ ]:
import plotly.express as px

fig = px.bar(df.query("type=='binary' | type=='scalar'"), x = "type", y = "performance_retention", color = "model_id", barmode="group", 
             facet_col = "task",
             title = "Performance retention on STS-B and Banking77 tasks",
       range_y=[df['performance_retention'].min()*0.99,df['performance_retention'].max()*1.01],
       category_orders={"type":["binary",  "scalar" ]}, 
       labels = {"performance_retention":"Performance retention", "type":"Method", "model_id":"Model"})

fig.add_hline(y=100)

fig.show()

In [ ]:
import plotly.express as px

fig = px.bar(df.query("type=='binary' |type=='ternary' |type=='scalar' "), 
             x = "type", y = "performance_retention", color = "model_id", barmode="group", 
       range_y=[df['performance_retention'].min()*0.99, df['performance_retention'].max()*1.01],
       category_orders={"type":["binary" "ternary",  "scalar"]}, 
       labels = {"performance_retention":"Performance retention", "type":"Method", "model_id":"Model"},
                    facet_col = "task",
             title = "Performance retention on STS-B and Banking77 tasks: ternary, binary & scalar")

fig.add_hline(y=100)

fig.show()

In [ ]:

retention_interval = { v['size_multiplier']:v['performance_retention'] for i,v in df.groupby('size_multiplier').agg({"performance_retention":lambda x: np.max(x)- np.min(x)}).reset_index().iterrows()}
retention_interval

In [ ]:
df["e"] = df.apply(lambda x: retention_interval[x["size_multiplier"]], axis = 1)

In [ ]:
#grouped = df.query("type=='classic' | type=='binary' |type=='ternary' |type=='scalar' ").groupby(["model_id", "type"]).agg({"performance_retention":np.mean, "size_multiplier":np.mean})
#grouped=grouped.reset_index()


import plotly.express as px

error_y=dict(type='percent', value=retention_interval.values(),visible=True)

fig = px.scatter(df, x = "size_multiplier", y = "performance_retention", 
                 color = "type", trendline="ols", trendline_scope="overall",
                 trendline_options=dict(log_x=True),
                 labels = {"performance_retention":"Performance retention", "size_multiplier":"Size multiplier", "model_id":"Model"},
                 title = "The trade-off: performance retention vs. size multiplier (with trend)",
                 facet_col="task")
#                 ,error_y = "e")


fig.show()


In [ ]:
import plotly.express as px

fig = px.scatter(df.query("type=='binary' |type=='ternary' |type=='scalar' "), 
             x = "size_multiplier", y = "performance_retention", color = "type",
       range_y=[df['performance_retention'].min()*0.99, df['performance_retention'].max()*1.01],
       category_orders={"type":["binary" "ternary",  "scalar"]}, 
       labels = {"performance_retention":"Performance retention", "type":"Method", "model_id":"Model"},
                    facet_col = "task",
             title = "Performance retention on STS-B and Banking77 tasks: ternary, binary & scalar")


fig.show()

Is rotation beneficial ?

In [ ]:
quantized_df = df.query("quantization==True").query("rotation==False")[["task", "model_id", "performance_retention"]]
quantized_rotated_df =df.query("quantization==True").query("rotation==True")[["task","model_id", "performance_retention"]].rename({"performance_retention":"performance_retention_rotated"}, axis = 1)
benchmark_df= quantized_df.merge(quantized_rotated_df, on = ["task","model_id"])
mask = benchmark_df['performance_retention_rotated'].values> benchmark_df['performance_retention'].values

proportion_higher = np.mean(mask)
average_improvement = np.mean(benchmark_df['performance_retention_rotated'][mask] - benchmark_df['performance_retention'][mask])

average_decrease = np.mean(benchmark_df['performance_retention_rotated'][~mask] - benchmark_df['performance_retention'][~mask])

proportion_higher, average_improvement, average_decrease


## SVD decomp

### Collecting data 

#### With titan Embed Text v1

In [ ]:
model_id = "amazon.titan-embed-text-v1"
Xbanking = banking_operator.matrixes[model_id].astype(np.float32)
Xsts =  sts_titan_operator.matrixes[model_id].astype(np.float32)
X=np.concatenate([Xbanking, Xsts])
X.shape

#### Or With Cohere Embed v3

In [ ]:
model_id = "cohere.embed-english-v3"
Xbanking = cohere_banking_operator.matrixes[model_id].astype(np.float32)
Xsts =  sts_cohere_operator.matrixes[model_id].astype(np.float32)
X=np.concatenate([Xbanking, Xsts])
X.shape

Instead of seeing the size reduction vs. accuracy, using unsigned integers was useful, now, in order to understand the decay rate, we value symetry. Therefore, transformations will be using _signed_ integers.

In [ ]:

#Quantile quantization
X33, X66 = np.quantile(X,1/3 ), np.quantile(X,2/3)
ternary = [ np.array([2 if e>X66 else (1 if e>X33 else 0) for e in x]).astype(np.uint8) for x in X  ]
binary = [np.array([1 if e>0 else 0  for e in x]).astype(np.uint8) for x in X]

Xmin, Xmax = X.min(), X.max()
range = (Xmax - Xmin) / 255

def scalar_quantize(x):
    scaled = (x - Xmin) / range
    return scaled.astype(np.uint8)

scalar = [scalar_quantize(x) for x in X]


## Test on SVD rotations

In [ ]:
U, S, Vh = np.linalg.svd(X, full_matrices=False)
Ub, Sb, Vhb = np.linalg.svd(binary, full_matrices=False)
Ut, St, Vht = np.linalg.svd(ternary, full_matrices=False)
Us, Ss, Vhs = np.linalg.svd(scalar, full_matrices=False)


In [ ]:
def get_decay_rate(S, max_dim=1024):
    Snorm = np.sort(S)[::-1]
    Snorm = Snorm / Snorm[0]  
    return Snorm[:max_dim]



In [ ]:
import matplotlib.pyplot as plt

plt.plot(get_decay_rate(S, max_dim=30), label = "classic")
plt.plot(get_decay_rate(Ss, max_dim=30), label = "scalar")
plt.plot(get_decay_rate(St, max_dim=30), label = "ternary")
plt.plot(get_decay_rate(Sb, max_dim=30), label = "binary")

plt.legend()
plt.title("Decay rate of singular values \n(Extracted from test set STS B+Banking77 Cohere Embed v3)")
plt.savefig("stsb-decayrate-cohere.png")

In [ ]:
np.linalg.norm(U - Ub, ord = "fro"), np.linalg.norm(U - Ut, ord = "fro"), np.linalg.norm(U - Us, ord = "fro")

In [ ]:
np.linalg.norm(Vh - Vhb, ord = "fro"),  np.linalg.norm(Vh - Vht, ord = "fro"), np.linalg.norm(Vh - Vhs, ord = "fro")

In [ ]:
Ux, Sx, Vx = np.linalg.svd(X, full_matrices=True)


In [ ]:
reconstructed = np.dot(U[:, :1536], Vh)
reconstructedbinary = np.dot(Ub[:, :1536], Vhb)
reconstructedternary = np.dot(Ut[:, :1536], Vht)
reconstructedscalar = np.dot(Us[:, :1536], Vhs)


In [ ]:
np.linalg.norm(X - reconstructedbinary, ord = "fro"), np.linalg.norm(X - reconstructedternary, ord = "fro"),  np.linalg.norm(X - reconstructedscalar, ord = "fro"), np.linalg.norm(X- reconstructed, ord = "fro")